In [23]:
import sys

In [24]:
# fetch from local path otherwise fetch from the remote path (if remote kernel is being used)
try: 
    sys.path.append('../../scripts')
    import database
    import utils
    import spark
except:
    try:
        sys.path.append('./scripts')
        import database
        import utils
        import spark
    except:
        raise RuntimeError('Failed to import from both local and remote paths. Program terminated.')

In [25]:
spark = spark.setup_spark_session(app_name='Dictionary')

In [26]:
db, mongo = database.setup_database()

In [27]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
# Define the schema
schema = StructType([
    StructField('words', StringType(), True),
    StructField('count', IntegerType() , True),
])

In [28]:
# Read the CSV file without header 
df = spark.read.csv('hdfs://localhost:54310/user/datascience/data/dict/part-00000',sep="\t", header=False, schema=schema)

In [29]:
df.toPandas().tail(100)

,words,count
359179,zwinki,13
359180,zwirl,1
359181,zwischenahn,1
359182,zwischendurch,1
359183,zwitter,1
359184,zwol,1
359185,zwoll,5
359186,zx,6
359187,zxcvasdfqwerti,1
359188,zxuilen,1


In [30]:
collection_name = "dictionary"

# Drop the existing collection if it exists
if collection_name in db.list_collection_names():
    db[collection_name].drop()

In [31]:
# Convert PySpark DataFrame to a list of dictionary where each dictionary represents a row in the DataFrame
# and the keys of the dictionary represent the column names
data = df.toPandas().to_dict(orient='records')

# print first 10 records to check the format
print(data[:3])

[{'words': 'a', 'count': 4}, {'words': 'aa', 'count': 190}, {'words': 'aaa', 'count': 99}]


In [32]:
# Insert the dictionary into the collection named "dictionary"
db[collection_name].insert_many(data)

In [22]:
# Check whether the data has been inserted into the collection
records =  db[collection_name].find().limit(4)
for record in db[collection_name].find().limit(4):


In [27]:
# Stop the spark session
spark.stop()

# Close the database connection
mongo.close()